# Step 1: Load Data

In [2]:
#import findspark
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import mllib
from pyspark import ml
#findspark.init("/opt/spark-2.4.0")
#sc = SparkContext('local')
#spark = SparkSession(sc)
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark import SparkConf, SparkContext
from numpy import array
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType
from pyspark.sql import *
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import udf
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import *

In [3]:
# File location and type
file_location = "/FileStore/tables/2015_data.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)


resident_status,education_1989_revision,education_2003_revision,education_reporting_flag,month_of_death,sex,detail_age_type,detail_age,age_substitution_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_decedents_status,marital_status,day_of_week_of_death,current_data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury_for_causes_w00_y34_except_y06_and_y07_,icd_code_10th_revision,358_cause_recode,113_cause_recode,130_infant_cause_recode,39_cause_recode,number_of_entity_axis_conditions,entity_condition_1,entity_condition_2,entity_condition_3,entity_condition_4,entity_condition_5,entity_condition_6,entity_condition_7,entity_condition_8,entity_condition_9,entity_condition_10,entity_condition_11,entity_condition_12,entity_condition_13,entity_condition_14,entity_condition_15,entity_condition_16,entity_condition_17,entity_condition_18,entity_condition_19,entity_condition_20,number_of_record_axis_conditions,record_condition_1,record_condition_2,record_condition_3,record_condition_4,record_condition_5,record_condition_6,record_condition_7,record_condition_8,record_condition_9,record_condition_10,record_condition_11,record_condition_12,record_condition_13,record_condition_14,record_condition_15,record_condition_16,record_condition_17,record_condition_18,record_condition_19,record_condition_20,race,bridged_race_flag,race_imputation_flag,race_recode_3,race_recode_5,hispanic_origin,hispanic_originrace_recode
1,null,3,1,01,M,1,084,null,42,22,10,null,1,M,1,2015,U,7,B,N,null,null,I500,230,067,null,22,02,11I500,61L031,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,02,I500,L031,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,01,null,null,1,1,100,6
1,null,6,1,01,M,1,070,null,40,20,09,null,1,M,2,2015,U,7,C,N,null,null,C349,093,027,null,08,08,11I469,21R042,31C349,41J189,42J90,61J449,62I251,63F179,null,null,null,null,null,null,null,null,null,null,null,null,07,C349,F179,I251,I469,J440,J90,R042,null,null,null,null,null,null,null,null,null,null,null,null,null,01,null,null,1,1,100,6
1,null,3,1,01,F,1,091,null,44,24,11,null,4,W,6,2015,U,7,C,N,null,null,G309,189,052,null,17,01,11G309,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,01,G309,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,01,null,null,1,1,100,6
1,null,3,1,01,F,1,040,null,34,14,06,null,4,S,5,2015,N,3,B,Y,9,9,X91,434,129,null,41,06,11T71,12X91,21T71,61T149,62S223,63S299,null,null,null,null,null,null,null,null,null,null,null,null,null,null,05,X91,S223,S299,T149,T71,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,03,null,null,2,3,100,8
1,null,5,1,01,F,1,089,null,43,23,11,null,6,W,5,2015,U,7,C,N,null,9,I250,214,062,null,21,03,11I250,21S720,61X590,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,03,I250,S720,X590,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,01,null,null,1,1,100,6
1,null,6,1,01,M,1,078,null,41,21,10,null,4,W,6,2015,U,7,C,N,null,null,I516,233,068,null,22,04,11I499,21I516,61E780,62F179,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,04,I516,E780,F179,I499,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,03,null,null,2,3,100,8
1,null,4,1,01,M,1,092,null,44,24,11,null,4,W,1,2015,U,7,R,N,null,null,E274,161,111,null,37,01,11E274,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,01,E274,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,01,null,null,1,1,100,6
1,null,3,1,01,M,1,082,null,42,22,10,null,4,M,1,2015,U,7,C,N,null,null,I500,230,067,null,22,01,11I500,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,01,I500,null,null,null,null,nul

# Step 2: Feature Selection

In [5]:
#Filter out column with no data 
names_test = df.columns
for name in names_test:
    print(name + "  ")
    print(df.select(name).dropna().count())
    
names = df.columns
names.remove("method_of_disposition")
names.append("method_of_disposition")
to_be_removed = ["race_imputation_flag", "bridged_race_flag", "record_condition_20", \
                 "record_condition_19", "record_condition_18", "record_condition_17", \
                 "record_condition_16", "record_condition_15", "record_condition_14", \
                 "record_condition_13", "record_condition_12", "record_condition_11", \
                 "record_condition_10", "record_condition_9", "record_condition_8", \
                 "record_condition_7", "record_condition_6", "record_condition_5", \
                 "record_condition_4", "record_condition_3", "record_condition_2", \
                 "record_condition_1", "130_infant_cause_recode", \
                 "education_1989_revision", "age_substitution_flag", "infant_age_recode_22", \
                 "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", \
                 "entity_condition_1", "entity_condition_2", "entity_condition_3", "entity_condition_4", \
                 "entity_condition_5", "entity_condition_6","entity_condition_7", "entity_condition_8", \
                 "entity_condition_9", "entity_condition_10", "entity_condition_11", "entity_condition_12", \
                 "entity_condition_13", "entity_condition_14","entity_condition_15",  "entity_condition_16", \
                 "entity_condition_17", "entity_condition_18","entity_condition_19", "entity_condition_20"]

#Remove columns of df which contains no data
for removed in to_be_removed:
        names.remove(removed)

df_feature_select = df.select(names)
df_feature_select = df_feature_select.dropna()
df_feature_select.show()

resident_status 
2718198
education_1989_revision 
73043
education_2003_revision 
2645155
education_reporting_flag 
2718198
month_of_death 
2718198
sex 
2718198
detail_age_type 
2718198
detail_age 
2718198
age_substitution_flag 
13
age_recode_52 
2718198
age_recode_27 
2718198
age_recode_12 
2718198
infant_age_recode_22 
23526
place_of_death_and_decedents_status 
2718198
marital_status 
2718198
day_of_week_of_death 
2718198
current_data_year 
2718198
injury_at_work 
2718198
manner_of_death 
2329834
method_of_disposition 
2718198
autopsy 
2718198
activity_code 
213679
place_of_injury_for_causes_w00_y34_except_y06_and_y07_ 
208215
icd_code_10th_revision 
2718198
358_cause_recode 
2718198
113_cause_recode 
2718198
130_infant_cause_recode 
23526
39_cause_recode 
2718198
number_of_entity_axis_conditions 
2718198
entity_condition_1 
2718198
entity_condition_2 
2116531
entity_condition_3 
1499128
entity_condition_4 
930069
entity_condition_5 
534655
entity_condition_6 
295511
entity_condition_7 
158881
entity_condition_8 
83901
entity_condition_9 
43955
entity_condition_10 
22372
entity_condition_11 
11050
entity_condition_12 
5199
entity_condition_13 
2178
entity_condition_14 
924
entity_condition_15 
3
entity_condition_16 
0
entity_condition_17 
0
entity_condition_18 
0
entity_condition_19 
0
entity_condition_20 
0
number_of_record_axis_conditions 
2718198
record_condition_1 
2718198
record_condition_2 
2078630
record_condition_3 
1415469
record_condition_4 
851506
record_condition_5 
466357
record_condition_6 
242718
record_condition_7 
122601
record_condition_8 
60394
record_condition_9 
29110
record_condition_10 
13454
record_condition_11 
5971
record_condition_12 
2409
record_condition_13 
767
record_condition_14 
182
record_condition_15 
6
record_condition_16 
0
record_condition_17 
0
record_condition_18 
0
record_condition_19 
0
record_condition_20 
0
race 
2718198
bridged_race_flag 
12288
race_imputation_flag 
26149
race_recode_3 
2718198
race_recode_5 
2718198
hispanic_origin 
2718198
hispanic_originrace_recode 
2718198
+---------------+-----------------------+------------------------+--------------+---+---------------+----------+-------------+-------------+-------------+-----------------------------------+--------------+--------------------+-----------------+--------------+---------------+-------+----------------------+----------------+----------------+---------------+--------------------------------+--------------------------------+----+-------------+-------------+---------------+--------------------------+---------------------+
resident_status|education_2003_revision|education_reporting_flag|month_of_death|sex|detail_age_type|detail_age|age_recode_52|age_recode_27|age_recode_12|place_of_death_and_decedents_status|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|autopsy|icd_code_10th_revision|358_cause_recode|113_cause_recode|39_cause_recode|number_of_entity_axis_conditions|number_of_record_axis_conditions|race|race_recode_3|race_recode_5|hispanic_origin|hispanic_originrace_recode|method_of_disposition|
+---------------+-----------------------+------------------------+--------------+---+---------------+----------+-------------+-------------+-------------+-----------------------------------+--------------+--------------------+-----------------+--------------+---------------+-------+----------------------+----------------+----------------+---------------+--------------------------------+--------------------------------+----+-------------+-------------+---------------+--------------------------+---------------------+
 1| 3| 1| 01| M| 1| 084| 42| 22| 10| 1| M| 1| 2015| U| 7| N| I500| 230| 067| 22| 02| 02| 01| 1| 1| 100| 6| B|
 1| 6| 1| 01| M| 1| 070| 40| 20| 09| 1| M| 2| 2015| U| 7| N| C349| 093| 027| 08| 08| 07| 01| 1| 1| 100| 6| C|
 1| 3| 1| 01| F| 1| 091| 44| 24| 11| 4| W| 6| 2015| U| 7| N| G309| 189| 052| 17| 01| 01| 01| 1| 1| 100| 6| C|
 1| 3| 1| 01| F| 1| 040| 34| 14| 06| 4| S| 5| 2015

In [6]:
##TRANSFORM STRING TO FLOAT
df_feature_select_dup = df_feature_select
for name in names:
    indexer = StringIndexer(inputCol=name, outputCol=name + "_index")
    df_feature_select_dup = indexer.fit(df_feature_select_dup).transform(df_feature_select_dup)
    
#names_all = df_feature_select_dup.columns
#print(names_all)

for name in names:
    df_feature_select_dup = df_feature_select_dup.drop(name)
    

df_feature_select_dup.columns


def transData(data):
    return data.rdd.map(lambda r: [r[-1], Vectors.dense(r[:-1])]). \
           toDF(['labelString','features'])
    
df_feature_select_trans=transData(df_feature_select_dup)
df_feature_select_trans.show()


+-----------+--------------------+
labelString| features|
+-----------+--------------------+
 1.0|[0.0,0.0,0.0,0.0,...|
 0.0|[0.0,4.0,0.0,0.0,...|
 0.0|[0.0,0.0,0.0,0.0,...|
 1.0|[0.0,0.0,0.0,0.0,...|
 0.0|[0.0,5.0,0.0,0.0,...|
 0.0|[0.0,4.0,0.0,0.0,...|
 2.0|[0.0,1.0,0.0,0.0,...|
 0.0|[0.0,0.0,0.0,0.0,...|
 1.0|[0.0,0.0,0.0,0.0,...|
 0.0|[0.0,6.0,0.0,0.0,...|
 1.0|[0.0,4.0,0.0,0.0,...|
 0.0|[0.0,0.0,0.0,0.0,...|
 0.0|[0.0,0.0,0.0,0.0,...|
 0.0|[0.0,4.0,0.0,0.0,...|
 0.0|[0.0,2.0,0.0,0.0,...|
 1.0|[0.0,0.0,0.0,0.0,...|
 1.0|[0.0,2.0,0.0,0.0,...|
 0.0|[0.0,0.0,0.0,0.0,...|
 0.0|[0.0,1.0,0.0,0.0,...|
 0.0|[0.0,4.0,0.0,0.0,...|
+-----------+--------------------+
only showing top 20 rows

#### Use ChiSqSelector to select top 10 features

In [8]:
selector = ChiSqSelector(numTopFeatures=10, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="labelString")
result = selector.fit(df_feature_select_trans).transform(df_feature_select_trans)

def extract(row):
    return (row.labelString, ) + tuple(float(x) for x in row.features.values) + tuple(float(x) for x in row.selectedFeatures.values)
result_featuers_df = result.rdd.map(extract).toDF(["labelString"])

#Convert to Panda dataframe
pd_df = result_featuers_df.toPandas()

#Compare values of original columns with values of selected top 10 columns
for i in range(2,30):
  for j in range(30,40):
    feature_col = "_" + str(i)
    selected_col = "_" + str(j)
    if(pd_df[feature_col].equals(pd_df[selected_col])):
      print(i)

2
3
5
6
7
8
9
10
11
12

### Step 3: Select Resident Status, Education, Sex, Age_recode_12, Marital_status, Month_of_death, Place_of_death as Selected Features

In [10]:
df = df['resident_status', 'education_2003_revision', 'sex', 'age_recode_12', 'marital_status', 'month_of_death','place_of_death_and_decedents_status','method_of_disposition']

rs = udf(lambda x: 1 if x == '2' else 2 if x == '3' else 3 if x == '4' else 0)
edu = udf(lambda x: 1 if x == '2' else 2 if x == '3' else 3 if x == '4' else 4 if x == '5' else 5 if x == '6' else 6 if x == '7' else 7 if x == '8' else 0)
se = udf(lambda x: 1 if x == 'F' else 0)
recode12 = udf(lambda x: 1 if x == '2' else 2 if x == '3' else 3 if x == '4' else 4 if x == '5' else 5 if x == '6' else 6 if x == '7' else 7 if x == '8' else 8 if x == '9' else 9 if x == '10' else 10 if x == '11' else 0)
mar = udf(lambda x: 1 if x == 'S' else 2 if x == 'W' else 3 if x == 'D' else 4 if x == 'M' else 0)
mon = udf(lambda x: 1 if x == '02' else 2 if x == '03' else 3 if x == '04' else 4 if x == '05' else 5 if x == '06' else 6 if x == '07' else 7 if x == '08' else 8 if x == '09' else 9 if x == '10' else 10 if x == '11' else 11 if x == '12' else 0)
pla = udf(lambda x: 1 if x == '2' else 2 if x == '3' else 3 if x == '4' else 4 if x == '5' else 5 if x == '6' else 6 if x == '7' else 7 if x == '9' else 0)
method = udf(lambda x: 1 if x == 'B' else 2 if x == 'C' else 0 )

#convert all Character category to integer category & Convert Column type to Double and 
df = df.withColumn("resident_status",rs(df['resident_status']).cast(DoubleType()))
df = df.withColumn("education_2003_revision",edu(df['education_2003_revision']).cast(DoubleType()))
df = df.withColumn("sex",se(df['sex']).cast(DoubleType()))
df = df.withColumn("age_recode_12",recode12(df['age_recode_12']).cast(DoubleType()))
df = df.withColumn("marital_status",mar(df['marital_status']).cast(DoubleType()))
df = df.withColumn("month_of_death",mon(df['month_of_death']).cast(DoubleType()))
df = df.withColumn("place_of_death_and_decedents_status",pla(df['place_of_death_and_decedents_status']).cast(DoubleType()))
df = df.withColumn("method_of_disposition",method(df['method_of_disposition']).cast(DoubleType()))
df = df.dropna()
df.show()


+---------------+-----------------------+---+-------------+--------------+--------------+-----------------------------------+---------------------+
resident_status|education_2003_revision|sex|age_recode_12|marital_status|month_of_death|place_of_death_and_decedents_status|method_of_disposition|
+---------------+-----------------------+---+-------------+--------------+--------------+-----------------------------------+---------------------+
 0.0| 2.0|0.0| 9.0| 4.0| 0.0| 0.0| 1.0|
 0.0| 5.0|0.0| 0.0| 4.0| 0.0| 0.0| 2.0|
 0.0| 2.0|1.0| 10.0| 2.0| 0.0| 3.0| 2.0|
 0.0| 2.0|1.0| 0.0| 1.0| 0.0| 3.0| 1.0|
 0.0| 4.0|1.0| 10.0| 2.0| 0.0| 5.0| 2.0|
 0.0| 5.0|0.0| 9.0| 2.0| 0.0| 3.0| 2.0|
 0.0| 3.0|0.0| 10.0| 2.0| 0.0| 3.0| 0.0|
 0.0| 2.0|0.0| 9.0| 4.0| 0.0| 3.0| 2.0|
 0.0| 2.0|1.0| 10.0| 2.0| 0.0| 3.0| 1.0|
 0.0| 0.0|1.0| 10.0| 2.0| 0.0| 5.0| 2.0|
 0.0| 5.0|1.0| 9.0| 4.0| 0.0| 5.0| 1.0|
 0.0| 2.0|1.0| 9.0| 4.0| 0.0| 5.0| 2.0|
 0.0| 2.0|1.0| 0.0| 4.0| 0.0| 0.0| 2.0|
 0.0| 5.0|0.0| 10.0| 2.0| 0.0| 5.0| 2.0|
 0.0| 1.0|0.0| 0.0| 4.0| 0.0| 5.0| 2.0|
 0.0| 2.0|0.0| 0.0| 1.0| 0.0| 0.0| 1.0|
 0.0| 1.0|0.0| 0.0| 1.0| 0.0| 6.0| 1.0|
 0.0| 2.0|0.0| 0.0| 4.0| 0.0| 3.0| 2.0|
 0.0| 3.0|1.0| 9.0| 2.0| 0.0| 5.0| 2.0|
 0.0| 5.0|0.0| 10.0| 2.0| 0.0| 5.0| 2.0|
+---------------+-----------------------+---+-------------+--------------+--------------+-----------------------------------+---------------------+
only showing top 20 rows

# Model 1:Decision Tree

In [12]:
(trainingData1, testData1) = df.randomSplit([0.7, 0.3])

#Create labeled point to fit into Decision tree model
def createdLabeledPoints(fields):
  resident_status = fields[0]
  education_2003_revision = fields[1]
  sex = fields[2]
  age_recode_12 = fields[3]
  marital_status = fields[4]
  month_of_death = fields[5]
  place_of_death_and_decedents_status = fields[6]
  method_of_disposition = fields[7]
  
  return LabeledPoint(method_of_disposition, array([resident_status, education_2003_revision, sex, age_recode_12, marital_status, month_of_death, place_of_death_and_decedents_status]))

#Convert training data to rdd inorder to fit into decision model
rdd = trainingData1.rdd
trainingRdd = rdd.map(createdLabeledPoints)

#create a Decision Tree model with max Depth 6 and Maxbins 79
model = DecisionTree.trainClassifier(trainingRdd, numClasses = 3, categoricalFeaturesInfo = {0:4, 1:9, 2:2, 3:12, 4:5, 5:12, 6:8}, impurity = 'gini', maxDepth = 7, maxBins = 79)

print(model.toDebugString())


DecisionTreeModel classifier of depth 7 with 253 nodes
 If (feature 3 in {0.0})
 If (feature 1 in {5.0,1.0,6.0,2.0,7.0,3.0,4.0})
 If (feature 4 in {3.0,0.0})
 If (feature 6 in {5.0,6.0,7.0,3.0,4.0})
 If (feature 0 in {1.0,0.0})
 If (feature 1 in {7.0,5.0,6.0,3.0})
 If (feature 0 in {0.0})
 Predict: 2.0
 Else (feature 0 not in {0.0})
 Predict: 2.0
 Else (feature 1 not in {7.0,5.0,6.0,3.0})
 If (feature 6 in {4.0,3.0})
 Predict: 2.0
 Else (feature 6 not in {4.0,3.0})
 Predict: 2.0
 Else (feature 0 not in {1.0,0.0})
 If (feature 6 in {3.0,4.0,6.0,7.0})
 If (feature 5 in {0.0,1.0,6.0,2.0,8.0,4.0})
 Predict: 2.0
 Else (feature 5 not in {0.0,1.0,6.0,2.0,8.0,4.0})
 Predict: 2.0
 Else (feature 6 not in {3.0,4.0,6.0,7.0})
 If (feature 1 in {5.0,1.0,6.0,2.0,7.0,4.0})
 Predict: 2.0
 Else (feature 1 not in {5.0,1.0,6.0,2.0,7.0,4.0})
 Predict: 2.0
 Else (feature 6 not in {5.0,6.0,7.0,3.0,4.0})
 If (feature 0 in {1.0,0.0})
 If (feature 0 in {0.0})
 If (feature 6 in {0.0})
 Predict: 2.0
 Else (feature 6 not in {0.0})
 Predict: 2.0
 Else (feature 0 not in {0.0})
 If (feature 1 in {7.0,3.0,6.0,5.0})
 Predict: 2.0
 Else (feature 1 not in {7.0,3.0,6.0,5.0})
 Predict: 2.0
 Else (feature 0 not in {1.0,0.0})
 If (feature 5 in {0.0,5.0,1.0,9.0,3.0,4.0})
 If (feature 1 in {7.0,6.0,3.0,5.0})
 Predict: 2.0
 Else (feature 1 not in {7.0,6.0,3.0,5.0})
 Predict: 2.0
 Else (feature 5 not in {0.0,5.0,1.0,9.0,3.0,4.0})
 If (feature 1 in {5.0,6.0,2.0,7.0,4.0})
 Predict: 2.0
 Else (feature 1 not in {5.0,6.0,2.0,7.0,4.0})
 Predict: 2.0
 Else (feature 4 not in {3.0,0.0})
 If (feature 6 in {3.0,4.0,5.0})
 If (feature 1 in {5.0,6.0,7.0,3.0,4.0})
 If (feature 2 in {0.0})
 If (feature 0 in {1.0,0.0})
 Predict: 2.0
 Else (feature 0 not in {1.0,0.0})
 Predict: 2.0
 Else (feature 2 not in {0.0})
 If (feature 1 in {7.0,3.0,5.0,6.0})
 Predict: 2.0
 Else (feature 1 not in {7.0,3.0,5.0,6.0})
 Predict: 2.0
 Else (feature 1 not in {5.0,6.0,7.0,3.0,4.0})
 If (feature 4 in {1.0})
 If (feature 2 in {0.0})
 Predict: 2.0
 Else (feature 2 not in {0.0})
 Predict: 2.0
 Else (feature 4 not in {1.0})
 If (feature 1 in {2.0})
 Predict: 2.0
 Else (feature 1 not in {2.0})
 Predict: 2.0
 Else (feature 6 not in {3.0,4.0,5.0})
 If (feature 0 in {1.0,0.0})
 If (feature 0 in {1.0})
 If (feature 1 in {1.0})
 Predict: 1.0
 Else (feature 1 not in {1.0})
 Predict: 2.0
 Else (feature 0 not in {1.0})
 If (feature 6 in {6.0,0.0})
 Predict: 2.0
 Else (feature 6 not in {6.0,0.0})
 Predict: 2.0
 Else (feature 0 not in {1.0,0.0})
 If (feature 6 in {6.0})
 If (feature 0 in {2.0})
 Predict: 2.0
 Else (feature 0 not in {2.0})
 Predict: 2.0
 Else (feature 6 not in {6.0})
 If (feature 1 in {7.0,6.0,5.0,3.0})
 Predict: 2.0
 Else (feature 1 not in {7.0,6.0,5.0,3.0})
 Predict: 2.0
 Else (feature 1 not in {5.0,1.0,6.0,2.0,7.0,3.0,4.0})
 If (feature 4 in {1.0,0.0})
 If (feature 6 in {0.0,5.0,1.0,2.0,3.0,4.0})
 If (feature 4 in {1.0})
 If (feature 6 in {4.0,2.0,0.0})
 If (feature 0 in {1.0,0.0})
 Predict: 2.0
 Else (feature 0 not in {1.0,0.0})
 Predict: 2.0
 Else (feature 6 not in {4.0,2.0,0.0})
 If (feature 6 in {1.0})
 Predict: 1.0
 Else (feature 6 not in {1.0})
 Predict: 2.0
 Else (feature 4 not in {1.0})
 If (feature 6 in {4.0})
 If (feature 0 in {2.0,1.0})
 Predict: 2.0
 Else (feature 0 not in {2.0,1.0})
 Predict: 2.0
 Else (feature 6 not in {4.0})
 If (feature 0 in {1.0,0.0})
 Predict: 2.0
 Else (feature 0 not in {1.0,0.0})
 Predict: 2.0
 Else (feature 6 not in {0.0,5.0,1.0,2.0,3.0,4.0})
 If (feature 4 in {0.0})
 If (feature 0 in {0.0,1.0,2.0})
 If (feature 5 in {8.0,10.0,7.0})
 Predict: 2.0
 Else (feature 5 not in {8.0,10.0,7.0})
 Predict: 2.0
 Else (feature 0 not in {0.0,1.0,2.0})
 If (feature 5 in {8.0,7.0,11.0,10.0})
 Predict: 0.0
 Else (feature 5 not in {8.0,7.0,11.0,10.0})
 Predict: 0.0
 Else (feature 4 not in {0.0})
 If (feature 0 in {1.0,0.0})
 If (feature 2 in {0.0})
 Predict: 1.0
 Else (feature 2 not in {0.0})
 Predict: 1.0
 Else (feature 0 not in {1.0,0.0})
 If (feature 5 in {4.0,9.0})
 Predict: 0.

####Test Decision Tree

In [14]:
testrdd = testData1.rdd
testingData = testrdd.map(createdLabeledPoints)
predictions = model.predict(testingData.map(lambda x: x.features))
labelsAndPredictions = testingData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda vp: vp[0] != vp[1]).count() /float(testingData.count())
print('Testing Error = ' + str(testErr))
print('Learned classification tree model:')
print(model)

Testing Error = 0.4464897218270127
Learned classification tree model:
DecisionTreeModel classifier of depth 7 with 253 nodes

###### Decision Tree model:  Accuracy of testing data: 56.36%

# Model 2: Multilayer Neuron Network

In [17]:
def transData(data):
    return data.rdd.map(lambda r: [r[-1], Vectors.dense(r[:-1])]).\
           toDF(['label','features'])
    
df2= transData(df)

(trainingData2, testData2) = df2.randomSplit([0.7, 0.3])
trainingData2=trainingData2["label","features"]
trainingData2.first()

# specify layers for the neural network:
# input layer of size 11 (features), two intermediate of size 5 and 4
# and output of size 7 (classes)
layers = [7, 10, 10, 15, 3, 7, 8, 12, 20, 3]

# create the trainer and set its parameters
FNN = MultilayerPerceptronClassifier(maxIter=200, layers=layers, blockSize=128, seed=1234)
model = FNN.fit(trainingData2)

testData2=testData2["label","features"]
testData2.first()

Out[ 109 ]: Row(label=0.0, features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]))

####Test Multilayer Neuron Network

In [19]:
# compute accuracy on the test set
result = model.transform(testData2)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 0.5407705011089302

###### Multi Layer Neuron Network:  Accuracy of testing data: 54.07%

# Model 3: Multinomial Logistic Regression

In [22]:
from pyspark.ml.classification import LogisticRegression

df_multi_log = df2["label","features"]
df_multi_log.show()

# Load training data
(trainingData3, testData3) = df_multi_log.randomSplit([0.7, 0.3])

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(trainingData3)

# Print the coefficients and intercept for multinomial logistic regression
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

trainingSummary = lrModel.summary

+-----+--------------------+
label| features|
+-----+--------------------+
 1.0|[0.0,2.0,0.0,9.0,...|
 2.0|[0.0,5.0,0.0,0.0,...|
 2.0|[0.0,2.0,1.0,10.0...|
 1.0|[0.0,2.0,1.0,0.0,...|
 2.0|[0.0,4.0,1.0,10.0...|
 2.0|[0.0,5.0,0.0,9.0,...|
 0.0|[0.0,3.0,0.0,10.0...|
 2.0|[0.0,2.0,0.0,9.0,...|
 1.0|[0.0,2.0,1.0,10.0...|
 2.0|[0.0,0.0,1.0,10.0...|
 1.0|[0.0,5.0,1.0,9.0,...|
 2.0|[0.0,2.0,1.0,9.0,...|
 2.0|[0.0,2.0,1.0,0.0,...|
 2.0|[0.0,5.0,0.0,10.0...|
 2.0|[0.0,1.0,0.0,0.0,...|
 1.0|[0.0,2.0,0.0,0.0,...|
 1.0|[0.0,1.0,0.0,0.0,...|
 2.0|[0.0,2.0,0.0,0.0,...|
 2.0|[0.0,3.0,1.0,9.0,...|
 2.0|[0.0,5.0,0.0,10.0...|
+-----+--------------------+
only showing top 20 rows

Coefficients: 
3 X 7 CSRMatrix

Intercept: [-1.13726202731,0.570672135784,0.566589891521]

####Test Multi Nominal Regression model

In [24]:
# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# for multiclass, we can inspect metrics on a per-label basis
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

objectiveHistory:
0.9220689666406373
False positive rate by label:
label 0: 0.0
label 1: 1.0
label 2: 0.0
True positive rate by label:
label 0: 0.0
label 1: 1.0
label 2: 0.0
Precision by label:
label 0: 0.0
label 1: 0.45931292362939685
label 2: 0.0
Recall by label:
label 0: 0.0
label 1: 1.0
label 2: 0.0
F-measure by label:
label 0: 0.0
label 1: 0.6294920249004012
label 2: 0.0
Accuracy: 0.45931292362939685
FPR: 0.45931292362939685
TPR: 0.45931292362939685
F-measure: 0.28913382235839236
Precision: 0.21096836181298412
Recall: 0.45931292362939685

In [25]:
predicted_result = lrModel.transform(testData3)
predicted_result
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predicted_result)
print(accuracy)

0.45823892319072956

###### Multinomial Regression model: Accuracy of training data: 45.93%;  Accuracy of testing data: 45.82%